In [1]:
import pandas as pd

DATA_PATH = 'data/'
PERIOD_LABEL = 'validation'

In [30]:
w_weights = [0.5, 0.5]
w_list = list()

for w_file in ['lgb_weights', 'tf_weights']:
    w_list.append(pd.read_csv(DATA_PATH + 'external/' + w_file + '_' + PERIOD_LABEL + '.csv')\
                  .query('id.str.contains("%s")' % PERIOD_LABEL).sort_values('id'))

final_weights = pd.concat([
    w_list[0]['id'].reset_index(drop=True),
    pd.DataFrame((w_list[0].iloc[:,1:]*w_weights[0]).values + (w_list[1].iloc[:,1:]*w_weights[1]).values,
                 columns=['F%s' % c for c in range(1,29)]).reset_index(drop=True)], axis=1)

In [33]:
final_weights.to_csv(DATA_PATH + 'external/' + 'final_weights' + '_' + PERIOD_LABEL + '.csv', index=False)